In [1]:
import pandas as pd
import datetime as dt

## Constructing Section 301 US Tariffs on China

### Tariff Actions

The key source of the US tariffs on china is this document:

[https://hts.usitc.gov/view/China%20Tariffs?release=2020HTSABasicB](https://hts.usitc.gov/view/China%20Tariffs?release=2020HTSABasicB)

which is essentially a master document of HS codes effected by the 301 investigation. The key issue is how to map this information into somethig that we would understand. **Note** that this ``.pdf`` file can be easily converted into excell (if you have Adobe DC) by simply cliking export as and select the Excel format. The output is pretty clean and can then be read in by pandas

In [2]:
file_path = "./data/China-Tariffs.xlsx"

In [3]:
df = pd.read_excel(file_path, skiprows = 6)

In [4]:
df.drop(labels = "Unnamed: 2", axis = 1, inplace = True)

df.columns = ["hs8", "Tariff_Action"]

df = df[df.hs8.str.contains("Harmonized Tariff Schedule 8-digit Subheading")==False]

df = df[df.hs8.str.contains("Harmonized Tariff Schedule 10-digit Number")==False]

df = df[~df.Tariff_Action.isnull()]

df.head()

,hs8,Tariff_Action
0,0101.21.00,9903.88.15
1,0101.29.00,9903.88.15
2,0101.30.00,9903.88.15
3,0101.90.30,9903.88.15
4,0101.90.40,9903.88.15


In [5]:
tariff_dict = dict(zip(list(df.Tariff_Action.unique()),[0,0,0,0,0,0]))

In [6]:
df[dt.datetime(2018,7,6)] = 0
df[dt.datetime(2018,8,23)] = 0
df[dt.datetime(2018,9,24)] = 0
df[dt.datetime(2019,6,1)] = 0
df[dt.datetime(2019,9,1)] = 0
df[dt.datetime(2020,2,14)] = 0

---
### MFN Rates

It's amazing how complicated/hard it is to sort this out is. Here is an overview of data resources [https://www.usitc.gov/data/index.htm](https://www.usitc.gov/data/index.htm) Then at this site [https://www.usitc.gov/tariff_affairs/documents/tariff_data/tariff_data_2017.zip](https://www.usitc.gov/tariff_affairs/documents/tariff_data/tariff_data_2017.zip) is a zip file with HS8 codes and then the tariffs that each products faces. Side note: what is amazing is how a lot of the duties have piece rate components + ad valorum (like Crucini's stuff). 

The choice here will be to take the MFN ad valorum rate and go with that. One issue that arises with this is that some products have a value assigned to be 10,000. These appear to be products that are subject to quotas which superseade things. Will have to deal with this some how.

Fortunatly, reading this in is straightforwad...

In [7]:
file_path = "./data/tariff_database_2017.xlsx"

In [8]:
mfn_df = pd.read_excel(file_path, usecols = ["hts8", "brief_description", "mfn_ad_val_rate"], dtype = {"hts8": str})

In [9]:
mfn_df.columns = ["hs8", "description", "mfn_rate"]

In [10]:
mfn_df["mfn_rate"] = 100*mfn_df["mfn_rate"]

In [11]:
mfn_df.head()

,hs8,description,mfn_rate
0,01012100,Live purebred breeding horses,0.0
1,01012900,Live horses other than purebred breeding horses,0.0
2,01013000,Live asses,6.8
3,01019030,Mules and hinnies imported for immediate slaug...,0.0
4,01019040,Mules and hinnies not imported for immediate s...,4.5


In [12]:
mfn_df.loc[mfn_df.mfn_rate > 350, "mfn_rate"] = 0

In [13]:
mfn_df.mfn_rate.max()

350.0

---
### Maping HTS actions into tariffs...

This took sometime to figure out. So in the China Tariff document, it has the following language:

"For reference, heading 9903.88.01 became effective on July 6, 2018, heading 9903.88.02 became effective on August 23, 2018, heading 9903.88.03 became effective on September 24, 2018, 9903.88.04 became effective on September 24, 2018, 9903.88.15 became effective on September 1, 2019, and 9903.88.16 is suspended pursuant to Federal Register notice of December 18, 2019 (84 FR 243)"

So this is how it is working. The Federal Register is publishing annexes undercertain headings, e.g. 9903.88.02. What is important, each one of these headings maps into a product code in the master document and the documents below describe the associated tariff rate and time of implementation. The US trade representative website has links to each register annoucment. Look here [https://ustr.gov/issue-areas/enforcement/section-301-investigations/tariff-actions](https://ustr.gov/issue-areas/enforcement/section-301-investigations/tariff-actions) which is the website associated with the 301 investigation into China.

Below, I tracked down each action and the associated tariff change as posted in the register. So here is the outline and then assignment of the tariff.

---
#### July 2018 Tariffs

**Action 9903.88.01** (Also known as the 34 of 60 billion list, tariff is +25 percent): This is the first list action that took place on July 6, 2018, described [here](https://ustr.gov/issue-areas/enforcement/section-301-investigations/section-301-china/34-billion-trade-action) then to see how this is mapped via the federal register open [this document outlining stuff](https://ustr.gov/sites/default/files/2018-13248.pdf)

In [14]:
tariff_dict.update({"9903.88.01": 25})

df[dt.datetime(2018,7,6)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.01": 0})

---
#### August 2018 Tariffs

**Action 9903.88.02** (This is the next 16 billion list, tariff is +25 percent): This is the next action taking place on [August 23, 2018](https://ustr.gov/issue-areas/enforcement/section-301-investigations/section-301-china/16-billion-trade-action) and again mapped via the federal register [here](https://ustr.gov/sites/default/files/enforcement/301Investigations/2018-17709.pdf)

In [15]:
tariff_dict.update({"9903.88.02": 25})

df[dt.datetime(2018,8,23)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.02": 0})

---
#### September 2018 (then June 2019) Tariffs

**Actions 9903.88.03, 9903.88.04** (This is the 200 billiion list +10 percent on September 24 2018, +15 percent on June 2019): These two actions are taken together. Morover, this list is subject to futher actions. The first discussion is on [September 2018](https://ustr.gov/issue-areas/enforcement/section-301-investigations/section-301-china/200-billion-trade-action) and the initial action is the 10 percent under these two headings described [here](https://ustr.gov/sites/default/files/enforcement/301Investigations/83%20FR%2047974.pdf). The website describes how things changed over time. The key change is that [here](https://ustr.gov/sites/default/files/enforcement/301Investigations/84_FR_20459.pdf) for the June 2019 increase of these tariffs from 10 to 25 percent.

In [16]:
tariff_dict.update({"9903.88.03": 10, "9903.88.04": 10})
# Looking through this, it appears the .04 action is at the 10 digit level, vs
# the stuff being at 8 digit. 

df[dt.datetime(2018,9,24)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.03": 0, "9903.88.04": 0})

In [17]:
tariff_dict.update({"9903.88.03": 15, "9903.88.04": 15})

df[dt.datetime(2019,6,1)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.03": 0, "9903.88.04": 0})

---

#### September 2019 (Febuary 2020) Tariffs

**Action 9903.88.15** (First part of the 300 billion list, +15 percent on September 1, 2019, 7.5 on): This was the big list that would cover eveything and annouced [here](https://ustr.gov/issue-areas/enforcement/section-301-investigations/section-301-china/300-billion-trade-action). [Initially it was at 10 percent](https://ustr.gov/sites/default/files/enforcement/301Investigations/Notice_of_Modification_%28List_4A_and_List_4B%29.pdf), but then went to [15 percent](https://ustr.gov/sites/default/files/enforcement/301Investigations/Notice_of_Modification%E2%80%93August_2019.pdf) and implemented on Steptember 2019. [These were then reduced to 7.5 percent](https://ustr.gov/sites/default/files/enforcement/301Investigations/Notice_of_Modification-January_2020.pdf) on January 2020

In [18]:
tariff_dict.update({"9903.88.15": 15})

df[dt.datetime(2019,9,1)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.15": 0})

In [19]:
tariff_dict.update({"9903.88.15": -7.5})

df[dt.datetime(2020,2,14)] = df["Tariff_Action"].map(tariff_dict)

tariff_dict.update({"9903.88.15": 0})

---
#### December 2019 Tariffs (Not implemented)

**9903.88.16** (Second part of the 300 billion list): This list was annouced with the ".15" list, but not to go into action untill December 18, 2019. See the back (page 141) of the [federal register notice](https://ustr.gov/sites/default/files/enforcement/301Investigations/Notice_of_Modification_%28List_4A_and_List_4B%29.pdf). [It was eventually suspended and not put in place.](https://ustr.gov/sites/default/files/enforcement/301Investigations/Notice_of_Modification%E2%80%93December_2019.pdf)

### Cleaning and Organizing

Now that we have the tariffs by time by product, we will just clean things up to conform with the way we want to use it later

In [20]:
df.drop(labels = "Tariff_Action", axis = 1, inplace = True)

df["hs8"] = df.hs8.str.replace(".","")

df["hs6"] = df.hs8.str[0:6]

df.head()

,hs8,2018-07-06 00:00:00,2018-08-23 00:00:00,2018-09-24 00:00:00,2019-06-01 00:00:00,2019-09-01 00:00:00,2020-02-14 00:00:00,hs6
0,01012100,0,0,0,0,15,-7.5,010121
1,01012900,0,0,0,0,15,-7.5,010129
2,01013000,0,0,0,0,15,-7.5,010130
3,01019030,0,0,0,0,15,-7.5,010190
4,01019040,0,0,0,0,15,-7.5,010190


Now let's merge this with the mfn rates so we have the initial starting point...

In [21]:
df = df.merge(mfn_df, left_on = "hs8", right_on = "hs8", how = "left")

In [22]:
df.rename(mapper = {"mfn_rate":dt.datetime(2018,1,1)}, axis = 1, inplace = True)

In [23]:
col_org = ["hs8", dt.datetime(2018,1,1), dt.datetime(2018,7,6), dt.datetime(2018,8,23), dt.datetime(2018,9,24), 
dt.datetime(2019,6,1), 
dt.datetime(2019,9,1), 
dt.datetime(2020,2,14), "hs6", "description"]

df = df[col_org]

In [24]:
df.head()

,hs8,2018-01-01 00:00:00,2018-07-06 00:00:00,2018-08-23 00:00:00,2018-09-24 00:00:00,2019-06-01 00:00:00,2019-09-01 00:00:00,2020-02-14 00:00:00,hs6,description
0,01012100,0.0,0,0,0,0,15,-7.5,010121,Live purebred breeding horses
1,01012900,0.0,0,0,0,0,15,-7.5,010129,Live horses other than purebred breeding horses
2,01013000,6.8,0,0,0,0,15,-7.5,010130,Live asses
3,01019030,0.0,0,0,0,0,15,-7.5,010190,Mules and hinnies imported for immediate slaug...
4,01019040,4.5,0,0,0,0,15,-7.5,010190,Mules and hinnies not imported for immediate s...


Now I will melt the dataframe to make it long. Groupby hs8 code and take the cummulative sum of the "value" in the dataframe. This will give the level of the tariff rate and how it changes by time.

In [25]:
df = df.melt(id_vars = ["hs8", "hs6", "description"])

df.rename(columns = {"variable": "time_of_tariff"}, inplace = True)

df['tariff'] = df.groupby(['hs8'])['value'].apply(lambda x: x.cumsum())

In [26]:
df.head()

,hs8,hs6,description,time_of_tariff,value,tariff
0,01012100,010121,Live purebred breeding horses,2018-01-01,0.0,0.0
1,01012900,010129,Live horses other than purebred breeding horses,2018-01-01,0.0,0.0
2,01013000,010130,Live asses,2018-01-01,6.8,6.8
3,01019030,010190,Mules and hinnies imported for immediate slaug...,2018-01-01,0.0,0.0
4,01019040,010190,Mules and hinnies not imported for immediate s...,2018-01-01,4.5,4.5


In [27]:
hs_grp = df.groupby(["hs6"])

hs_grp.get_group(("010130"))

,hs8,hs6,description,time_of_tariff,value,tariff
2,01013000,010130,Live asses,2018-01-01,6.8,6.8
10445,01013000,010130,Live asses,2018-07-06,0.0,6.8
20888,01013000,010130,Live asses,2018-08-23,0.0,6.8
31331,01013000,010130,Live asses,2018-09-24,0.0,6.8
41774,01013000,010130,Live asses,2019-06-01,0.0,6.8
52217,01013000,010130,Live asses,2019-09-01,15.0,21.8
62660,01013000,010130,Live asses,2020-02-14,-7.5,14.3


Choices. Now the issue is that we want to go down to the 6 level. So the solution will be the following. We will try a couple of different aggregations and see if it matters. Note that it appears that the Tariff Retaliation was at the 6 digit level (as there is no variation in tariffs across products within the 6 digit level). What variation this is missing is the initial level which (sometimes) does vary within products. 

In [28]:
us_tariffs_hs6 = df.groupby(["hs6", "time_of_tariff"]).agg({"tariff": "max", "description": "first"})

In [29]:
us_tariffs_hs6.reset_index(inplace = True)

In [30]:
us_tariffs_hs6[us_tariffs_hs6.hs6 == "020329"]

,hs6,time_of_tariff,tariff,description
315,020329,2018-01-01,0.0,"Frozen retail cuts of meat of swine, nesi"
316,020329,2018-07-06,0.0,"Frozen retail cuts of meat of swine, nesi"
317,020329,2018-08-23,0.0,"Frozen retail cuts of meat of swine, nesi"
318,020329,2018-09-24,10.0,"Frozen retail cuts of meat of swine, nesi"
319,020329,2019-06-01,25.0,"Frozen retail cuts of meat of swine, nesi"
320,020329,2019-09-01,25.0,"Frozen retail cuts of meat of swine, nesi"
321,020329,2020-02-14,25.0,"Frozen retail cuts of meat of swine, nesi"


In [31]:
us_tariffs_hs6.groupby(["time_of_tariff"]).tariff.median()

time_of_tariff
2018-01-01     2.7
2018-07-06     3.7
2018-08-23     3.7
2018-09-24    12.5
2019-06-01    25.0
2019-09-01    25.0
2020-02-14    25.0
Name: tariff, dtype: float64

In [32]:
us_tariffs_hs6.to_csv(".\\data"+ "\\us_tariffs_2020.csv",index = False)

In [35]:
test = pd.read_csv(".\\data"+ "\\us_tariffs_2020.csv", dtype = {"hs6": str})

In [36]:
test.head()

,hs6,time_of_tariff,tariff,description
0,010121,2018-01-01,0.0,Live purebred breeding horses
1,010121,2018-07-06,0.0,Live purebred breeding horses
2,010121,2018-08-23,0.0,Live purebred breeding horses
3,010121,2018-09-24,0.0,Live purebred breeding horses
4,010121,2019-06-01,0.0,Live purebred breeding horses
